In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
from lab_coffee_utils import load_coffee_data

In [3]:
#load data
X,Y = load_coffee_data()
print(X.shape, Y.shape)

(200, 2) (200, 1)


In [4]:
#normalize data
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)

In [7]:
#tiling 
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape) 

(200000, 2) (200000, 1)


In [21]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 3)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)


In [24]:
model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10


6250/6250 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.3564
Epoch 2/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 0.1208
Epoch 3/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.1119
Epoch 4/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.1070
Epoch 5/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.0263
Epoch 6/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.0134
Epoch 7/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0091
Epoch 8/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0064
Epoch 9/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0046
Epoch 10/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0034


In [25]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
#these params are to be used later for infernce in np forward prop
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[-9.67e-03  1.55e+01 -1.02e+01]
 [-9.21e+00  1.29e+01 -1.78e-01]] 
b1: [-11.54   2.04 -11.15]
W2:
 [[-40.5 ]
 [-36.14]
 [-43.25]] 
b2: [23.14]


In [26]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
predictions = 
 [[9.67e-01]
 [2.17e-06]]
decisions = 
[[1]
 [0]]


In [27]:
def sigmoid(z):
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))
    return g

In [39]:
def my_dense(A_in, W, B):
    units = W.shape[1]
    A_out= np.zeros(units)
    # for j in range(units):
    #     w = W[:,j]
    #     z = np.dot(w,a_in)+b[j]
    #     a_out[j] = sigmoid(z)  replaced by vectorized implementation
    Z = np.matmul(A_in,W)+B
    A_out= sigmoid(Z)
    return A_out
        

In [40]:
def my_sequential(x ,W1,b1,W2,b2):
    a1 = my_dense(x,W1,b1)
    a2 = my_dense(a1,W2,b2)
    return a2

In [41]:
def my_predict(X,W1,b1,W2,b2):
    m = X.shape[0]
    p = np.zeros((m,1))
    for i in range(m):
        p[i,0] = my_sequential(X[i],W1,b1,W2,b2)
    return p


In [42]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1, b1, W2, b2)

C:\Users\Asjad1\AppData\Local\Temp\ipykernel_6012\1851064077.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p[i,0] = my_sequential(X[i],W1,b1,W2,b2)


In [43]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]
